In [134]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import tensorflow as tf
import pathlib
import PIL
import time
import zipfile
import random
from tensorflow import keras as ks
from tensorflow.keras.layers import *
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [135]:
train_df=pd.read_csv("../input/trailblazers-south-african-qualification-challenge/Train (5).csv")
test_df=pd.read_csv("../input/trailblazers-south-african-qualification-challenge/Test (7).csv")
sub=pd.read_csv("../input/trailblazers-south-african-qualification-challenge/SampleSubmission (3).csv")

In [136]:
Id=test_df["Place_ID X Date"]

In [137]:
train_df.head()

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,...,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,...,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,...,49.839714,-78.342701,34.296977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,010Q650 X 2020-01-05,2020-01-05,010Q650,49.0,10.0,55.0,1113.67,96,6.911948,21.300001,...,29.181258,-73.896588,30.545446,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,010Q650 X 2020-01-06,2020-01-06,010Q650,21.0,9.0,52.0,1164.82,95,13.900001,44.700001,...,0.797294,-68.612480,26.899694,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
L=[i for i in train_df.columns if i not in test_df.columns]
L

['target', 'target_min', 'target_max', 'target_variance', 'target_count']

In [139]:
train_df["target"].nunique()

375

In [140]:
train_df.isna().sum().sort_values(ascending =False).head(20)

L3_CH4_solar_zenith_angle                              24765
L3_CH4_solar_azimuth_angle                             24765
L3_CH4_sensor_zenith_angle                             24765
L3_CH4_sensor_azimuth_angle                            24765
L3_CH4_aerosol_optical_depth                           24765
L3_CH4_aerosol_height                                  24765
L3_CH4_CH4_column_volume_mixing_ratio_dry_air          24765
L3_NO2_tropospheric_NO2_column_number_density           8669
L3_HCHO_tropospheric_HCHO_column_number_density_amf     7425
L3_HCHO_tropospheric_HCHO_column_number_density         7425
L3_HCHO_solar_zenith_angle                              7425
L3_HCHO_solar_azimuth_angle                             7425
L3_HCHO_sensor_zenith_angle                             7425
L3_HCHO_sensor_azimuth_angle                            7425
L3_HCHO_cloud_fraction                                  7425
L3_HCHO_HCHO_slant_column_number_density                7425
L3_SO2_absorbing_aerosol

In [141]:
train_df.isna().sum().sort_values(ascending =False).head(20)/len(train_df)

L3_CH4_solar_zenith_angle                              0.810453
L3_CH4_solar_azimuth_angle                             0.810453
L3_CH4_sensor_zenith_angle                             0.810453
L3_CH4_sensor_azimuth_angle                            0.810453
L3_CH4_aerosol_optical_depth                           0.810453
L3_CH4_aerosol_height                                  0.810453
L3_CH4_CH4_column_volume_mixing_ratio_dry_air          0.810453
L3_NO2_tropospheric_NO2_column_number_density          0.283699
L3_HCHO_tropospheric_HCHO_column_number_density_amf    0.242989
L3_HCHO_tropospheric_HCHO_column_number_density        0.242989
L3_HCHO_solar_zenith_angle                             0.242989
L3_HCHO_solar_azimuth_angle                            0.242989
L3_HCHO_sensor_zenith_angle                            0.242989
L3_HCHO_sensor_azimuth_angle                           0.242989
L3_HCHO_cloud_fraction                                 0.242989
L3_HCHO_HCHO_slant_column_number_density

In [142]:
train_df.drop(["L3_CH4_solar_zenith_angle","L3_CH4_solar_azimuth_angle","L3_CH4_sensor_zenith_angle","L3_CH4_sensor_azimuth_angle","L3_CH4_aerosol_optical_depth",
             "L3_CH4_aerosol_height","L3_CH4_CH4_column_volume_mixing_ratio_dry_air","Place_ID X Date","Place_ID"], axis=1, inplace=True)

In [143]:
train_df.shape

(30557, 73)

In [144]:
import datetime as dt
train_df["Date"]=pd.to_datetime(train_df["Date"])
train_df['Date_year'] =train_df["Date"].dt.year
train_df['Date_month'] = train_df["Date"].dt.month
train_df['Date_day'] = train_df["Date"].dt.day
train_df.drop("Date", axis=1, inplace=True)

In [145]:
df_train=train_df.drop("target",axis=1)

In [146]:
test_df.drop(["L3_CH4_solar_zenith_angle","L3_CH4_solar_azimuth_angle","L3_CH4_sensor_zenith_angle","L3_CH4_sensor_azimuth_angle","L3_CH4_aerosol_optical_depth",
             "L3_CH4_aerosol_height","L3_CH4_CH4_column_volume_mixing_ratio_dry_air","Place_ID X Date","Place_ID"], axis=1, inplace=True)

test_df["Date"]=pd.to_datetime(test_df["Date"])
test_df['Date_year'] =test_df["Date"].dt.year
test_df['Date_month'] = test_df["Date"].dt.month
test_df['Date_day'] = test_df["Date"].dt.day
test_df.drop("Date", axis=1, inplace=True)

In [147]:
df_test=test_df

In [148]:
y_train = train_df[["target"]]

In [149]:
df_train.isna().sum()

target_min                                 0
target_max                                 0
target_variance                            0
target_count                               0
precipitable_water_entire_atmosphere       0
                                        ... 
L3_SO2_solar_azimuth_angle              7237
L3_SO2_solar_zenith_angle               7237
Date_year                                  0
Date_month                                 0
Date_day                                   0
Length: 74, dtype: int64

In [150]:
import random
missing_values = df_train.columns[df_train.isna().any()].to_list()
for each in missing_values:
    if (df_train[each].dtypes =='float64'):
        minimum= int(df_train[each].quantile(0.25))
        maximum= int(df_train[each].quantile(0.75))
        A=df_train[df_train[each].isnull()].index.tolist()
        for i in A:
            df_train.loc[i,each]=random.randint(minimum,maximum)
        df_train[each]=pd.to_numeric(df_train[each])
   

    elif(df_train[each].dtypes == 'object'):
        if ('True' in str(df_train[each].str.contains('No').unique().tolist())):
            df_train[each].fillna('No',inplace=True)
        elif('True' in str(df_train[each].str.contains('None').unique().tolist())):
            df_train[each].fillna('None',inplace=True)
        elif('True' in str(df_train[each].str.contains('Unf').unique().tolist())):
            df_train[each].fillna('Unf',inplace=True)
        else:
            A=df_train[df_train[each].isnull()].index.tolist()
            unique = df_train[each].unique().tolist()
            unique=pd.Series(unique).dropna().tolist()
            for i in A:
                df_train.loc[i,each]=random.choice(unique)


missing_values = df_test.columns[df_test.isna().any()].to_list()
for each in missing_values:
    if (df_test[each].dtypes =='float64'):
        minimum= int(df_test[each].quantile(0.25))
        maximum= int(df_test[each].quantile(0.75))
        A=df_test[df_test[each].isnull()].index.tolist()
        for i in A:
            df_test.loc[i,each]=random.randint(minimum,maximum)
        df_test[each]=pd.to_numeric(df_test[each])
   

    elif(df_test[each].dtypes == 'object'):
        if ('True' in str(df_test[each].str.contains('No').unique().tolist())):
            df_test[each].fillna('No',inplace=True)
        elif('True' in str(df_test[each].str.contains('None').unique().tolist())):
            df_test[each].fillna('None',inplace=True)
        elif('True' in str(df_test[each].str.contains('Unf').unique().tolist())):
            df_test[each].fillna('Unf',inplace=True)
        else:
            A=df_test[df_test[each].isnull()].index.tolist()
            unique = df_test[each].unique().tolist()
            unique=pd.Series(unique).dropna().tolist()
            for i in A:
                df_test.loc[i,each]=random.choice(unique)

In [151]:
df_train.drop(['target_min', 'target_max', 'target_variance', 'target_count'], axis=1, inplace=True)

In [152]:
df_train.shape, df_test.shape

((30557, 70), (16136, 70))

In [153]:
df_train.describe()

,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,...,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,Date_year,Date_month,Date_day
count,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,...,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.0,30557.000000,30557.000000
mean,15.302326,70.552747,0.006004,9.321342,0.416886,0.088486,0.000087,0.000152,-1.052364,0.258096,...,0.000024,-1.838045,0.163228,-13.727004,35.530751,-127.923983,46.513098,2020.0,2.095755,15.254312
std,10.688573,18.807884,0.003787,9.343226,2.707990,2.641924,0.000094,0.000113,0.621990,0.299712,...,0.000147,0.690757,0.165263,70.367161,17.229199,63.638963,13.116896,0.0,0.899081,8.921199
min,0.420044,5.128572,0.000139,-34.647879,-15.559646,-15.023647,-0.000051,0.000000,-3.941742,0.000000,...,-0.004309,-4.833706,0.000000,-111.997123,0.000000,-179.880630,0.000000,2020.0,1.000000,1.000000
25%,7.666667,58.600002,0.003403,3.123071,-1.097864,-1.415701,0.000050,0.000113,-1.487721,0.014184,...,-0.000020,-2.233243,0.000000,-95.143869,22.910227,-163.569653,38.363953,2020.0,1.000000,7.000000
50%,12.200000,74.099998,0.004912,8.478424,0.222092,0.012847,0.000072,0.000146,-1.110044,0.118137,...,0.000000,-1.963388,0.114964,-14.153469,37.000000,-152.621328,47.000000,2020.0,2.000000,15.000000
75%,19.900000,85.450001,0.007562,16.201563,1.772925,1.555559,0.000107,0.000185,-0.701402,0.435056,...,0.000071,-1.369259,0.295409,67.438383,49.784480,-127.000000,55.264808,2020.0,3.000000,23.000000
max,72.599998,100.000000,0.021615,37.437921,17.955124,15.055530,0.002998,0.002431,4.824802,1.000000,...,0.006988,3.736877,0.618176,80.544067,66.111289,179.776125,79.631711,2020.0,4.000000,31.000000


In [154]:
col_tr=[i for i in df_train.columns]
col_ts=[i for i in df_test.columns]


In [155]:
# def cal_rmsd_numpy(coord_1, coord_2):
#     rmsd = np.sqrt(((coord_1 - coord_2) ** 2).mean())    ## this would be the formula
#     return rmsd


In [156]:
def calcul_midofiedCVI(list):
    diff_nn_interval= np.diff(list)
    sd1=np.sqrt(np.std(diff_nn_interval , ddof=1)**2 *2 *0.5)
    sd2= np.sqrt(2*np.std(list , ddof=1) ** 2 -0.5 *np.std(\
              diff_nn_interval , ddof=1)**2)
    
    L=4*sd1
    T=4 *sd2
    return L **2 /T

In [157]:
df_train.describe()

,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,...,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,Date_year,Date_month,Date_day
count,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,...,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.0,30557.000000,30557.000000
mean,15.302326,70.552747,0.006004,9.321342,0.416886,0.088486,0.000087,0.000152,-1.052364,0.258096,...,0.000024,-1.838045,0.163228,-13.727004,35.530751,-127.923983,46.513098,2020.0,2.095755,15.254312
std,10.688573,18.807884,0.003787,9.343226,2.707990,2.641924,0.000094,0.000113,0.621990,0.299712,...,0.000147,0.690757,0.165263,70.367161,17.229199,63.638963,13.116896,0.0,0.899081,8.921199
min,0.420044,5.128572,0.000139,-34.647879,-15.559646,-15.023647,-0.000051,0.000000,-3.941742,0.000000,...,-0.004309,-4.833706,0.000000,-111.997123,0.000000,-179.880630,0.000000,2020.0,1.000000,1.000000
25%,7.666667,58.600002,0.003403,3.123071,-1.097864,-1.415701,0.000050,0.000113,-1.487721,0.014184,...,-0.000020,-2.233243,0.000000,-95.143869,22.910227,-163.569653,38.363953,2020.0,1.000000,7.000000
50%,12.200000,74.099998,0.004912,8.478424,0.222092,0.012847,0.000072,0.000146,-1.110044,0.118137,...,0.000000,-1.963388,0.114964,-14.153469,37.000000,-152.621328,47.000000,2020.0,2.000000,15.000000
75%,19.900000,85.450001,0.007562,16.201563,1.772925,1.555559,0.000107,0.000185,-0.701402,0.435056,...,0.000071,-1.369259,0.295409,67.438383,49.784480,-127.000000,55.264808,2020.0,3.000000,23.000000
max,72.599998,100.000000,0.021615,37.437921,17.955124,15.055530,0.002998,0.002431,4.824802,1.000000,...,0.006988,3.736877,0.618176,80.544067,66.111289,179.776125,79.631711,2020.0,4.000000,31.000000


In [158]:
# cols=df_train.columns.to_list()

In [159]:
# df_train["modified"]=df_train[cols].apply(lambda row: calcul_midofiedCVI(row[cols], axis=1))

In [173]:
df_train["mean"]=df_train.mean(axis=1)
df_train["count"]=df_train.count(axis=1)
df_train["std"]=df_train.std(axis=1)
df_train["min"]=df_train.min(axis=1)
df_train["max"]=df_train.max(axis=1)
df_train["kurt"]=df_train.kurt(axis=1)
df_train["skew"]=df_train.skew(axis=1)
df_train["q1"]=df_train.quantile(q=0.1,axis=1)
df_train["q2"]=df_train.quantile(q=0.2,axis=1)
df_train["q25"]=df_train.quantile(q=0.25,axis=1)
df_train["q3"]=df_train.quantile(q=0.3,axis=1)
df_train["q4"]=df_train.quantile(q=0.4,axis=1)
df_train["q5"]=df_train.quantile(q=0.5,axis=1)
df_train["q6"]=df_train.quantile(q=0.6,axis=1)
df_train["q7"]=df_train.quantile(q=0.7,axis=1)
df_train["q75"]=df_train.quantile(q=0.75,axis=1)
df_train["q8"]=df_train.quantile(q=0.8,axis=1)
df_train["q9"]=df_train.quantile(q=0.9,axis=1)

df_train=df_train.drop(columns=col_tr)

In [174]:
df_test["mean"]=df_test.mean(axis=1)
df_test["count"]=df_test.count(axis=1)
df_test["std"]=df_test.std(axis=1)
df_test["min"]=df_test.min(axis=1)
df_test["max"]=df_test.max(axis=1)
df_test["kurt"]=df_test.kurt(axis=1)
df_test["skew"]=df_test.skew(axis=1)
df_test["q1"]=df_test.quantile(q=0.1,axis=1)
df_test["q2"]=df_test.quantile(q=0.2,axis=1)
df_test["q25"]=df_test.quantile(q=0.25,axis=1)
df_test["q3"]=df_test.quantile(q=0.3,axis=1)
df_test["q4"]=df_test.quantile(q=0.4,axis=1)
df_test["q5"]=df_test.quantile(q=0.5,axis=1)
df_test["q6"]=df_test.quantile(q=0.6,axis=1)
df_test["q7"]=df_test.quantile(q=0.7,axis=1)
df_test["q75"]=df_test.quantile(q=0.75,axis=1)
df_test["q8"]=df_test.quantile(q=0.8,axis=1)
df_test["q9"]=df_test.quantile(q=0.9,axis=1)
df_test=df_test.drop(columns=col_ts)

In [175]:
df_test.shape, df_train.shape 

((16136, 18), (30557, 18))

In [176]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train = mms.fit_transform(df_train)
X_test = mms.transform(df_test)

In [177]:
y_train = mms.fit_transform(y_train)

In [178]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_train, y_train, test_size=0.1)

In [179]:
# from sklearn.decomposition import PCA
# pca = PCA()
# rd_xtrain = pca.fit_transform(xtrain)
# rd_xtest = pca.transform(xtest)

In [180]:
# x_test=pca.fit_transform(X_test)[:,:15]

In [181]:
# explained_variance=pca.explained_variance_ratio_
# explained_variance.shape

In [182]:
# with plt.style.context('dark_background'):
#     plt.figure(figsize=(12, 6))

#     plt.bar(range(85), explained_variance, alpha=0.6, align='center',label='individual explained variance')
#     plt.ylabel('Explained variance ratio')
#     plt.xlabel('Principal components')
#     plt.legend(loc='best')
#     plt.tight_layout()

In [183]:
# xtrain_ = rd_xtrain[:, :15]
# xtest_ = rd_xtest[:, :15]

In [184]:
# xtrain_.shape, xtest_.shape

In [185]:
# from sklearn.ensemble import RandomForestRegressor
# Rf = RandomForestRegressor(min_samples_split=15)
# Rf.fit(xtrain, ytrain)
# ypred=Rf.predict(xtest)

In [186]:
# from sklearn.metrics import mean_squared_error
# mean_squared_error(ytest, ypred)

In [187]:
# Rf.fit(xtest, ytest)

In [188]:
# y_pred_original4=y_pred_original4.tolist()
# y_pred_original4=[i[0] for i in y_pred_original4]
# submission = pd.DataFrame({'Place_ID X Date	': Id,'target': y_pred_original4})
# submission.to_csv('submission20.csv', index=False)

****

****

In [189]:
from sklearn import  ensemble
params = {
    "n_estimators": 500,
    "max_depth": 8,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(xtrain, ytrain)
mse = mean_squared_error(ytest, reg.predict(xtest))
mse

0.00238455933432019

In [190]:
mse

0.00238455933432019

In [ ]:
reg.fit(X_train, y_train)
y_pred4 = reg.predict(X_test) 
y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
y_pred_original4=y_pred_original4.tolist()
y_pred_original4=[i[0] for i in y_pred_original4]
submission = pd.DataFrame({'Place_ID X Date	': Id,'target': y_pred_original4})
submission.to_csv('submission11.csv', index=False)

****

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(objective='reg:squarederror')
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, xtrain, ytrain, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('MSE: %.5f (%53f)' % (mean(n_scores), std(n_scores)))

In [ ]:
model.fit(xtrain, ytrain)
yhat = model.predict(xtest)

In [ ]:
mean_squared_error(ytest, yhat)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred4 = model.predict(X_test) 
y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
y_pred_original4=y_pred_original4.tolist()
y_pred_original4=[i[0] for i in y_pred_original4]
submission = pd.DataFrame({'Place_ID X Date	': Id,'target': y_pred_original4})
submission.to_csv('submission12.csv', index=False)

****

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
model1 = LGBMRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model1, xtrain, ytrain, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.5f (%.5f)' % (mean(n_scores), std(n_scores)))


In [ ]:
model1 = LGBMRegressor()
model1.fit(xtrain, ytrain)
yhatt = model1.predict(xtest)
mean_squared_error(ytest, yhatt)

In [ ]:
model1.fit(X_train, y_train)

In [ ]:
y_pred4 = model1.predict(X_test) 
y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
y_pred_original4=y_pred_original4.tolist()
y_pred_original4=[i[0] for i in y_pred_original4]
submission = pd.DataFrame({'Place_ID X Date	': Id,'target': y_pred_original4})
submission.to_csv('submission13.csv', index=False)

In [ ]:
mean_squared_error(ytest, yhat)